## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/sample_libsvm_data.txt"
file_type = "txt"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "sample_libsvm_data_txt"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `sample_libsvm_data_txt`

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "sample_libsvm_data_txt"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
import pyspark

In [0]:
data=spark.read.format('libsvm').load("/FileStore/tables/sample_libsvm_data.txt")

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(692,[127,128,129...|
 1.0|(692,[158,159,160...|
 1.0|(692,[124,125,126...|
 1.0|(692,[152,153,154...|
 1.0|(692,[151,152,153...|
 0.0|(692,[129,130,131...|
 1.0|(692,[158,159,160...|
 1.0|(692,[99,100,101,...|
 0.0|(692,[154,155,156...|
 0.0|(692,[127,128,129...|
 1.0|(692,[154,155,156...|
 0.0|(692,[153,154,155...|
 0.0|(692,[151,152,153...|
 1.0|(692,[129,130,131...|
 0.0|(692,[154,155,156...|
 1.0|(692,[150,151,152...|
 0.0|(692,[124,125,126...|
 0.0|(692,[152,153,154...|
 1.0|(692,[97,98,99,12...|
 1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows

In [0]:
log_model=LogisticRegression()

In [0]:
fit_log_model=log_model.fit(data)

In [0]:
log_summary=fit_log_model.summary

In [0]:
log_summary.predictions

Out[9]: DataFrame[label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [0]:
lr_train,lr_test=data.randomSplit([0.7,0.3])

In [0]:
final_model=LogisticRegression()

In [0]:
fit_final=final_model.fit(lr_train)

In [0]:
predictions_label=fit_final.evaluate(lr_test)

In [0]:
predictions_label.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(692,[123,124,125...|[35.6252699653943...|[0.99999999999999...| 0.0|
 0.0|(692,[123,124,125...|[19.4488343581929...|[0.99999999642333...| 0.0|
 0.0|(692,[124,125,126...|[33.6189635768236...|[0.99999999999999...| 0.0|
 0.0|(692,[124,125,126...|[26.9830326588104...|[0.99999999999808...| 0.0|
 0.0|(692,[126,127,128...|[34.2029927479527...|[0.99999999999999...| 0.0|
 0.0|(692,[126,127,128...|[25.1864892330973...|[0.99999999998847...| 0.0|
 0.0|(692,[150,151,152...|[28.2797721197563...|[0.99999999999947...| 0.0|
 0.0|(692,[152,153,154...|[35.3310220295696...|[0.99999999999999...| 0.0|
 0.0|(692,[153,154,155...|[30.2157467621252...|[0.99999999999992...| 0.0|
 0.0|(692,[155,156,180...|[23.7295037982318...|[0.99999999995052...| 0.0|
 0.0|(692,[181,182,183...|[13.5145538644838...|[0.99999864885099...| 0.0|
 0.0|(692,[234,235,237...|[3.84842719263194...|[0.97913154256987...| 0.0|
 1.0|(692,[123,124,125...|[-25.712335049336...|[6.81200206938386...| 1.0|
 1.0|(692,[124,125,126...|[-16.381467697161...|[7.68455883951867...| 1.0|
 1.0|(692,[126,127,128...|[-22.138680091343...|[2.42825001227187...| 1.0|
 1.0|(692,[128,129,130...|[-25.496106953067...|[8.45632060927211...| 1.0|
 1.0|(692,[128,129,130...|[-27.160348061912...|[1.60107143821349...| 1.0|
 1.0|(692,[129,130,131...|[-25.227567296738...|[1.10613142613932...| 1.0|
 1.0|(692,[129,130,131...|[-32.392978667004...|[8.54885818063717...| 1.0|
 1.0|(692,[129,130,131...|[-19.424112438913...|[3.66619153498818...| 1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [0]:
my_eval=BinaryClassificationEvaluator()

In [0]:
my_final_roc=my_eval.evaluate(predictions_label.predictions)

In [0]:
my_final_roc

Out[18]: 1.0